In [2]:
import pandas as pd
from urllib.request import urlopen
import json
from datetime import datetime

In [3]:
# Connect to MongoDB
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
mongodb = client.chessDB
gameCollection = mongodb.gameCollection
playerCollection = mongodb.playerCollection

In [4]:
# get wins losses and draws for a data_json
def get_wld (data_json):
  
  wins1=losses1=draws1=wins2=losses2=draws2=wins3=losses3=draws3=0
  # try:    
  #   wins1,losses1,draws1= data_json["chess_rapid"]["record"].values()
  #   wins2,losses2,draws2= data_json["chess_bullet"]["record"].values()
  #   wins3,losses3,draws3= data_json["chess_blitz"]["record"].values() 
  # except:
  #   pass
  try:
    rapid_record = data_json.get("chess_rapid", {}).get("record", {})
    wins1,losses1,draws1 = rapid_record["win"] or 0, rapid_record["loss"] or 0, rapid_record["draw"] or 0

  except KeyError as e:
    pass

  try:
    bullet_record = data_json.get("chess_bullet", {}).get("record", {})
    wins2, losses2, draws2 = bullet_record.get("win", 0), bullet_record.get("loss", 0), bullet_record.get("draw", 0)
  except KeyError as e:
    pass

  try:
    blitz_record = data_json.get("chess_blitz", {}).get("record", {})
    wins3, losses3, draws3 = blitz_record.get("win", 0), blitz_record.get("loss", 0), blitz_record.get("draw", 0)
  except KeyError as e:
    pass
  # print(f"{wins3},{losses3},{draws3}")

  wins = wins1+wins2+wins3
  losses= losses1+losses2+losses3
  draws = draws1+draws2+draws3

  return wins,losses,draws

In [5]:
def getDate(data_json):
    
    try:
        dates = [data_json["chess_blitz"]["last"]["date"], data_json["chess_bullet"]["last"]["date"], data_json["chess_rapid"]["last"]["date"]]
        unixDate = max(dates)
        date = datetime.datetime.fromtimestamp(unixDate).strftime('%Y.%m.%d')
    except:
        date = datetime.today().strftime('%Y.%m.%d')

    return date

In [6]:
titles = ["GM", "WGM", "IM", "WIM", "FM", "WFM", "NM", "WNM", "CM", "WCM"]
names = []
playerDocuments = playerCollection.find()
all_players = {}
allNames = []
for doc in playerDocuments:
    try:
        allNames.append(doc['name'])
    except:
        print(doc)

for title in titles:
  url = "https://api.chess.com/pub/titled/{}".format(title)
  response = urlopen(url)
  data_json = json.loads(response.read())
  names.extend(data_json["players"])

players={}
chess_rapid_rating = chess_bullet_rating = chess_blitz_rating = 0
rd_blitz = rd_bullet = rd_rapid = 0
weight_rapid = weight_bullet = weight_blitz = 0

for id in names:
    url_name = "https://api.chess.com/pub/player/{}/stats".format(id)
    try:
        response = urlopen(url_name)
        data_json = json.loads(response.read())
    except:
        print(url_name)
   
    try:
        rd_rapid = data_json["chess_rapid"]["last"]["rd"]
        weight_rapid = (1/rd_rapid**2)
        chess_rapid_rating = data_json["chess_rapid"]["last"]["rating"]
    except:
        rd_rapid = 0
        weight_rapid = 0
        chess_rapid_rating = 0
  
    try:
        rd_bullet = data_json["chess_bullet"]["last"]["rd"]
        weight_bullet = (1/rd_bullet**2)
        chess_bullet_rating = data_json["chess_bullet"]["last"]["rating"]
    except:
        rd_bullet = 0
        weight_bullet = 0
        chess_bullet_rating = 0
  
    try:
        rd_blitz = data_json["chess_blitz"]["last"]["rd"]
        weight_blitz = (1/rd_blitz**2)
        chess_blitz_rating = data_json["chess_blitz"]["last"]["rating"]
    except:
        rd_blitz = 0
        weight_blitz = 0
        chess_blitz_rating = 0
    
    try:
        rating = int(((chess_rapid_rating*weight_rapid) + (chess_bullet_rating*weight_bullet) + (chess_blitz_rating*weight_blitz))/(weight_rapid+weight_blitz+weight_bullet))
    except:
        continue
    
    wins, losses, draws = get_wld(data_json)
    date = getDate(data_json)
    # print(wins, losses, draws, date, id, name)
    

    url_name = "https://api.chess.com/pub/player/{}".format(id)
    response = urlopen(url_name)
    data_json = json.loads(response.read())
    
    try:
        name = data_json['name']
        all_players[name] = {'wins':wins,'losses':losses,'draws':draws,'date':date,'id':id}
    except:
        continue
    
    if name in allNames:
    # update wins losses or draws
        myquery = {"name": name}
        myPlayer = playerCollection.find_one(myquery)
        if myPlayer is None:
            continue
    
        newvalues = { "$set": { "wins": wins, "losses": losses, "draws": draws } }
        playerCollection.update_one(myquery, newvalues)

        #update date
        newDate = myPlayer['dates']
        newDate.append(date)
        newvalues = { "$set": { "dates": newDate } }
        playerCollection.update_one(myquery, newvalues)

        #update rating
        newRating = myPlayer['ratings']
        newRating.append(rating)
        newvalues = { "$set": { "ratings": newRating} }
        playerCollection.update_one(myquery, newvalues)

    else:
        playerData = {'name': name,
                    'wins': wins,
                    'losses': losses,
                    'draws': draws,
                    'dates':[date],
                    'ratings':[rating]
                    }
    
        playerCollection.insert_one(playerData)
        allNames.append(name)


https://api.chess.com/pub/player/airgun1/stats


In [ ]:
print(all_players)

NameError: name 'all_players' is not defined

In [ ]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast
app = Flask(__name__)
api = Api(app)

In [ ]:
class Users(Resource):
    # methods go here
    def get(self):
        print('data',all_players)
        return {'data': all_players}, 200  # return data and 200 OK code
    
#     pass

api.add_resource(Users, '/users')  # '/users' is our entry point

In [ ]:
if __name__ == '__main__':
    app.run()  # run our Flask app